In [1]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import healpy as hp
import scipy.stats as st


In [2]:
def cal_power_spectrum_mc(lmax,D,theta): #power law, exclude the monopole later on, mc==model clean

    Al_mc = np.zeros(lmax+1)
    for l in range(lmax+1):
        Al_mc[l] = D * ((l)**theta)          # calculate model Als according to a power law
    return Al_mc

def getlm(lmax):#(include -m terms), outputs arrays of all l and m for an lmax
    halflen=hp.Alm.getsize(lmax)
    l1,m1=hp.Alm.getlm(lmax,np.arange(halflen))
    l2=np.flip(l1[m1>0])
    m2=np.flip(-m1[m1>0])
    l=np.concatenate((l2,l1))
    m=np.concatenate((m2,m1))
    return l,m

def getlmidx(lmax,ll): #input: lmax, the l that we want to find all (l,m) index
    mcount=lmax*(lmax+1)/2
    lmidx=hp.Alm.getidx(lmax,ll,np.arange(ll+1))
    truelmidx=np.concatenate((np.flip(mcount-(lmidx[1:]-lmax)),lmidx+mcount))
    truelmidx=truelmidx.astype(int)
    return truelmidx
#both definitions above from Kate

def getlmidx_v2(lmax,ll, mm): #input: lmax, the l and m that we want to find single (l,m) index
    l,m = getlm(lmax)
    for i in range((lmax+1)**2):
        if ll==l[i] and mm==m[i]:
            return i

In [3]:
#sped up with help from Daniel, 
#draws gw and em spherical harmonic coefficients from gw,em,gwem angular power spectra
#clean space output


def draw_alms_blms_v4(Al, Bl, Cl, lmax):
    l, m = hp.Alm.getlm(lmax)
    size = (lmax + 1) ** 2

    # Prepare output arrays
    alms = np.zeros(size, dtype=complex)
    blms = np.zeros(size, dtype=complex)

    # Identify indices for real and imaginary components
    real_mask = l > 0
    imag_mask = m > 0

    lr, mr = l[real_mask], m[real_mask]
    li, mi = l[imag_mask], m[imag_mask]

    # Precompute covariance matrices
    def compute_cov(A, B, C, half=False):
        factor = 0.5 if half else 1.0
        return [[factor * A, factor * C], [factor * C, factor * B]]

    # Generate real components
    for i, (ll, mm) in enumerate(zip(lr, mr)):
        half = mm != 0
        cov_r = compute_cov(Al[ll], Bl[ll], Cl[ll], half)
        ar, br = np.random.multivariate_normal([0, 0], cov_r)
        ai = bi = 0

        # Handle imaginary components if applicable
        if mm > 0:
            cov_i = compute_cov(Al[ll], Bl[ll], Cl[ll], True)
            ai, bi = np.random.multivariate_normal([0, 0], cov_i)

        # Assign values for +m
        idx = getlmidx_v2(lmax, ll, mm)
        alms[idx] = complex(ar, ai)
        blms[idx] = complex(br, bi)

        # Assign conjugate values for -m
        if mm != 0:
            idx_neg = getlmidx_v2(lmax, ll, -mm)
            sign = (-1) ** mm
            alms[idx_neg] = np.conj(alms[idx]) * sign
            blms[idx_neg] = np.conj(blms[idx]) * sign

    return alms, blms



In [4]:


#From Kate, expand the complex covariance matrix into 2x2 size larger real matrix

#FIXED to include m=0 exceptions
def C2Rcov(covmat,lmax): #input: complex covariance matrix
    size=len(covmat)
#     lmax=int(size**0.5) -1
    ll,mm=getlm(lmax)
    Rcovmat=np.zeros((2*size,2*size))
    for ii in range(size):   # -m
        for jj in range(size):
            p=ll[ii]
            q=mm[ii]
            l=ll[jj]
            m=mm[jj]
            if q==0:
                iip=ii #(p,-q) is (p,q) itself when q=0
            elif q!=0: #(p,-q) index <-> (p,q) index
                iip=size-1-ii
            if m==0:
                jjp=jj #(l,-m) is (l,m) itself when m=0
            elif m!=0:
                jjp=size-1-jj #(l,-m) index <-> (l,m) index
            #R_pq R_lm
            Rcovmat[ii][jj]=0.25*(covmat[ii][jj] + ((-1)**abs(q))*covmat[iip][jj] + ((-1)**abs(m))*covmat[ii][jjp] + ((-1)**abs(q+m))*covmat[iip][jjp])
            #I_pq I_lm
            Rcovmat[ii+size][jj+size]=0.25*(covmat[ii][jj] - ((-1)**abs(q))*covmat[iip][jj] - ((-1)**abs(m))*covmat[ii][jjp] + ((-1)**abs(q+m))*covmat[iip][jjp])
            #R_pq I_lm
            Rcovmat[ii][jj+size]=0.25*(-1j)*(covmat[ii][jj] + ((-1)**abs(q))*covmat[iip][jj] - ((-1)**abs(m))*covmat[ii][jjp] - ((-1)**abs(q+m))*covmat[iip][jjp])
            #I_pq R_lm
            Rcovmat[ii+size][jj]=0.25*(1j)*(covmat[ii][jj] - ((-1)**abs(q))*covmat[iip][jj] + ((-1)**abs(m))*covmat[ii][jjp] - ((-1)**abs(q+m))*covmat[iip][jjp])                
    return Rcovmat

#From Kate, altered for my specific case
def covcrosscl(blm,gwcov,lmax): #input: blm's, GW alm cov mat, output: Cl covmat
    Rgwcov=C2Rcov(gwcov,lmax) #take the NxN complex covariance matrix and create a 2Nx2N Real covariance matrix
    #blm=fullalm(blm)
    size=len(gwcov)
    #lmax=hp.Alm.getlmax(len(alm_gc))
    covcl=np.zeros((lmax+1,lmax+1))
    for l1 in range(lmax+1):
        lmidx1=getlmidx(lmax,l1)
        blm_re1=np.real(blm[lmidx1])
        blm_im1=np.imag(blm[lmidx1])
        A1=blm_re1/(2.*l1+1.)
        B1=blm_im1/(2.*l1+1.)
        idx1=np.concatenate((lmidx1,lmidx1+size))
        C=np.concatenate((A1,B1))
        for l2 in range(lmax+1):
            lmidx2=getlmidx(lmax,l2)
            blm_re2=np.real(blm[lmidx2])
            blm_im2=np.imag(blm[lmidx2])
            A2=blm_re2/(2.*l2+1.)
            B2=blm_im2/(2.*l2+1.)
            idx2=np.concatenate((lmidx2,lmidx2+size))
            D=np.concatenate((A2,B2))
            SS=Rgwcov[np.ix_(idx1,idx2)]
            covcl[l1][l2]=np.matmul(np.matmul(C,SS),
                                np.transpose(D))
    return covcl

def constructCrossPower(alms,blms,lmax):
    crossPower = np.zeros(lmax+1)
    for ll in range(lmax+1):
        idx = getlmidx(lmax,ll)
        crossPower[ll] = np.real((1/(2*ll+1))*np.sum(np.conj(alms[idx])*blms[idx]))
    return crossPower

#From Kate, expand the complex covariance matrix into 2x2 size larger real matrix

#FIXED to include m=0 exceptions
def C2Rcov(covmat,lmax): #input: complex covariance matrix
    size=len(covmat)
#     lmax=int(size**0.5) -1
    ll,mm=getlm(lmax)
    Rcovmat=np.zeros((2*size,2*size))
    for ii in range(size):   # -m
        for jj in range(size):
            p=ll[ii]
            q=mm[ii]
            l=ll[jj]
            m=mm[jj]
            if q==0:
                iip=ii #(p,-q) is (p,q) itself when q=0
            elif q!=0: #(p,-q) index <-> (p,q) index
                iip=size-1-ii
            if m==0:
                jjp=jj #(l,-m) is (l,m) itself when m=0
            elif m!=0:
                jjp=size-1-jj #(l,-m) index <-> (l,m) index
            #R_pq R_lm
            Rcovmat[ii][jj]=0.25*(covmat[ii][jj] + ((-1)**abs(q))*covmat[iip][jj] + ((-1)**abs(m))*covmat[ii][jjp] + ((-1)**abs(q+m))*covmat[iip][jjp])
            #I_pq I_lm
            Rcovmat[ii+size][jj+size]=0.25*(covmat[ii][jj] - ((-1)**abs(q))*covmat[iip][jj] - ((-1)**abs(m))*covmat[ii][jjp] + ((-1)**abs(q+m))*covmat[iip][jjp])
            #R_pq I_lm
            Rcovmat[ii][jj+size]=0.25*(-1j)*(covmat[ii][jj] + ((-1)**abs(q))*covmat[iip][jj] - ((-1)**abs(m))*covmat[ii][jjp] - ((-1)**abs(q+m))*covmat[iip][jjp])
            #I_pq R_lm
            Rcovmat[ii+size][jj]=0.25*(1j)*(covmat[ii][jj] - ((-1)**abs(q))*covmat[iip][jj] + ((-1)**abs(m))*covmat[ii][jjp] - ((-1)**abs(q+m))*covmat[iip][jjp])                
    return Rcovmat

#From Kate, altered for me specific case
def covcrosscl(blm,gwcov,lmax): #input: blm's, GW alm cov mat
    Rgwcov=C2Rcov(gwcov,lmax) #take the NxN complex covariance matrix and create a 2Nx2N Real covariance matrix
    #blm=fullalm(blm)
    size=len(gwcov)
    #lmax=hp.Alm.getlmax(len(alm_gc))
    covcl=np.zeros((lmax+1,lmax+1))
    for l1 in range(lmax+1):
        lmidx1=getlmidx(lmax,l1)
        blm_re1=np.real(blm[lmidx1])
        blm_im1=np.imag(blm[lmidx1])
        A1=blm_re1/(2.*l1+1.)
        B1=blm_im1/(2.*l1+1.)
        idx1=np.concatenate((lmidx1,lmidx1+size))
        C=np.concatenate((A1,B1))
        for l2 in range(lmax+1):
            lmidx2=getlmidx(lmax,l2)
            blm_re2=np.real(blm[lmidx2])
            blm_im2=np.imag(blm[lmidx2])
            A2=blm_re2/(2.*l2+1.)
            B2=blm_im2/(2.*l2+1.)
            idx2=np.concatenate((lmidx2,lmidx2+size))
            D=np.concatenate((A2,B2))
            SS=Rgwcov[np.ix_(idx1,idx2)]
            covcl[l1][l2]=np.matmul(np.matmul(C,SS),
                                np.transpose(D))
    return covcl

def constructCrossPower(alms,blms,lmax):
    crossPower = np.zeros(lmax+1)
    for ll in range(lmax+1):
        idx = getlmidx(lmax,ll)
        crossPower[ll] = np.real((1/(2*ll+1))*np.sum(np.conj(alms[idx])*blms[idx]))
    return crossPower

def getDirtyModelCrossPower(amplitude, exponent, gw_powerModelClean, em_powerModelClean, fisher, lmax):
    crossPowerModelClean= cal_power_spectrum_mc(lmax,amplitude,exponent)
    alms_modelClean, blms_modelClean = draw_alms_blms_v4(gw_powerModelClean, em_powerModelClean, crossPowerModelClean, lmax)
    alms_modelDirty = np.matmul(fisher,alms_modelClean)
    blms_modelDirty = np.matmul(fisher,blms_modelClean)
    crossPowerModelDirty = constructCrossPower(alms_modelDirty, blms_modelDirty,lmax)
    return crossPowerModelDirty, blms_modelDirty

def getNoiseFromCovmat(covmat, lmax): #specifically for spherical harmonic coefficients
    real_covmat = C2Rcov(covmat, lmax)
    means = np.zeros(2*((lmax+1)**2))
    rv = np.random.multivariate_normal(mean=means, cov=real_covmat)
    
    real_alms = rv[:(lmax+1)**2]
    imaginary_alms = rv[(lmax+1)**2:]
    
    noise_alms = np.zeros((lmax+1)**2, dtype=complex)
    for i in range((lmax+1)**2):
        noise_alms[i] = complex(real_alms[i], imaginary_alms[i])
    
    return noise_alms

def getDirtyInjectedCrossPower(gw_powerModelClean, em_powerModelClean, crossPowerSignal, fisher, gw_noise, lmax):
    alms_modelClean, blms_modelClean = draw_alms_blms_v4(gw_powerModelClean, em_powerModelClean, crossPowerSignal, lmax)
    alms_modelDirty, blms_modelDirty = np.matmul(fisher,alms_modelClean), np.matmul(fisher,blms_modelClean)
    alms_injectedDirty = alms_modelDirty + gw_noise
    crossPowerInjectedDirty = constructCrossPower(alms_injectedDirty, blms_modelDirty, lmax)
    return crossPowerInjectedDirty

def getDirtyModelCrossPowerMeanCovmatTable(amplitude_range, step_size, exponent, gw_powerModelClean, em_powerModelClean, fisher, lmax, nsim):
    amplitude_list = np.array(amplitude_range[0])
    val = amplitude_range[0]
    while val<amplitude_range[1]:
        val += step_size
        amplitude_list = np.append(amplitude_list,val)
        
    crossPowerModelDirty_all = np.zeros([len(amplitude_list), nsim, lmax+1])
    crossPowerModelDirty_means = np.zeros([len(amplitude_list), lmax+1])
    crossPowerModelDirty_std = np.zeros([len(amplitude_list), lmax+1])
    crossPowerModelDirty_sigma = np.zeros([len(amplitude_list), lmax+1, lmax+1])
    for i in range(len(amplitude_list)):
        for n in range(nsim):
            crossPowerModelDirty,_ = getDirtyModelCrossPower(amplitude_list[i], exponent, gw_powerModelClean, em_powerModelClean, fisher, lmax)
            crossPowerModelDirty_all[i,n] = crossPowerModelDirty
    
        crossPowerModelDirty_means[i] = np.mean(crossPowerModelDirty_all[i],axis=0)
        crossPowerModelDirty_std[i] = np.std(crossPowerModelDirty_all[i], axis=0)
        crossPowerModelDirty_sigma[i] = np.cov(crossPowerModelDirty_all[i],rowvar=False)
        
#         for ll in range(lmax+1):
#             crossPowerModelDirty_means[i,ll] = np.mean(crossPowerModelDirty_all[i,:,ll])
#             crossPowerModelDirty_std[i,ll] = np.std(crossPowerModelDirty_all[i,:,ll])
#             for pp in range(lmax+1):
#                 crossPowerModelDirty_sigma[i,ll,pp] = np.mean(crossPowerModelDirty_all[i,:,ll]*crossPowerModelDirty_all[i,:,pp]) - np.mean(crossPowerModelDirty_all[i,:,ll])*np.mean(crossPowerModelDirty_all[i,:,pp])
                
    return crossPowerModelDirty_means, crossPowerModelDirty_std, crossPowerModelDirty_sigma

In [5]:
########## Set parameters ######################
l_max = 6 #max angular resolution
nsim = 1000 #number of times we draw the alms/blms from the model
exp = 1 # exponent in model 

#calculate model clean gw power spectrum
gw_amp = 5e-97
Al_mc = cal_power_spectrum_mc(l_max,gw_amp,exp)

#calculate model clean em power spectrum
em_amp = 8e-101
Bl_mc = cal_power_spectrum_mc(l_max,em_amp,exp)

#calculate model clean gwem power spectrum amp limits
gwem_amp_max = np.sqrt(gw_amp*em_amp) # goes from inverse correlation (rho=-1) to correlation (rho=1)
amp_range = [-gwem_amp_max, gwem_amp_max]

num_steps = 202
step_size = (amp_range[1]-amp_range[0])/num_steps


############## Draw Models into Dirty Space ###########

#import Fisher matrix
Fisher = np.load('data/gw_fisher_lmax6_allfreq_HL.npy')

#draw model into dirty space once
Cl_md,_ = getDirtyModelCrossPower(gwem_amp_max, exp, Al_mc, Bl_mc, Fisher, l_max)

#get table of models in the dirty space at each amplitude interested
Cl_md_means, Cl_md_stds, Cl_md_sigmas = getDirtyModelCrossPowerMeanCovmatTable(amp_range, step_size, exp, Al_mc, Bl_mc, Fisher, l_max, nsim)

np.save("output/ClMeanDraw.npy", Cl_md_means)
np.save("output/ClStdDraw.npy", Cl_md_stds)
np.save("output/ClSigmaDraw.npy", Cl_md_sigmas)

############ Inject Signal into Dirty Space ##########
#calculate model clean cross-power signal
gwem_signal = 3e-99
Cl_signal = cal_power_spectrum_mc(l_max,gwem_signal,exp)

#calculate gw noise to add to alms (in dirty space)
gw_noise = getNoiseFromCovmat(Fisher, l_max)

#draw signal into dirty space + add O3 noise
Cl_injd = getDirtyInjectedCrossPower(Al_mc, Bl_mc, Cl_signal, Fisher, gw_noise, l_max)

############ Parameter Estimation ####################


/var/folders/63/39kkwt414rv8gx91d72hdt240000gn/T/ipykernel_92599/1585571098.py:91: ComplexWarning: Casting complex values to real discards the imaginary part
  Rcovmat[ii][jj]=0.25*(covmat[ii][jj] + ((-1)**abs(q))*covmat[iip][jj] + ((-1)**abs(m))*covmat[ii][jjp] + ((-1)**abs(q+m))*covmat[iip][jjp])
/var/folders/63/39kkwt414rv8gx91d72hdt240000gn/T/ipykernel_92599/1585571098.py:93: ComplexWarning: Casting complex values to real discards the imaginary part
  Rcovmat[ii+size][jj+size]=0.25*(covmat[ii][jj] - ((-1)**abs(q))*covmat[iip][jj] - ((-1)**abs(m))*covmat[ii][jjp] + ((-1)**abs(q+m))*covmat[iip][jjp])
/var/folders/63/39kkwt414rv8gx91d72hdt240000gn/T/ipykernel_92599/1585571098.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  Rcovmat[ii][jj+size]=0.25*(-1j)*(covmat[ii][jj] + ((-1)**abs(q))*covmat[iip][jj] - ((-1)**abs(m))*covmat[ii][jjp] - ((-1)**abs(q+m))*covmat[iip][jjp])
/var/folders/63/39kkwt414rv8gx91d72hdt240000gn/T/ipykernel_92599/1585571098.py: